# 멜론에서 장르별 스터디 셀러 크롤링

## 필요한 라이브러리 임포트

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm

import re
import pandas as pd
import time as time

from pymongo import MongoClient
from pymongo import CursorType

## 함수 정의

In [2]:
#   page의 주소가 1, 51, 101로 구분되어서 만들어주었다.
# 시작 위치
music_url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0400&steadyYn=Y'
# Page 넘기는 주소
np_url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0400&steadyYn=Y#params%5BgnrCode%5D=GN0400&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=Y&po=pageObj&startIndex={}"


# 노래 제목 반환
def find_title(driver):
    title =  driver.find_elements_by_class_name('ellipsis.rank01')
    title2 = []
    for i in title:
        title2.append(i.text)
        
    return title2
    

    
# 가수 이름 반환
def find_singer(driver):
    singer = driver.find_elements_by_class_name('ellipsis.rank02')
    singer2 = []
    for i in singer:
        singer2.append(i.text)
    
    return singer2
    

    
# 크롤링 후 페이지 넘김
def next_page(driver,np_url ,page):
    p_url = np_url.format(page)
    page_url = p_url
    driver.get(page_url)
    time.sleep(3)
      
        
    
# 노래 정보가 들어있는 주소 반환    
def find_song_address(driver):
    song_address = driver.find_elements_by_class_name("input_check")
    song_address_li = []
    for i in song_address:
        song_address_li.append(i.get_attribute("value"))
    time.sleep(3)
    
    return song_address_li



# 노래의 가사 반환
def lyric_crawl(driver, song_address_li):

    LYRIC = []
    
    for address in song_address_li[1:]:
        try:
            driver.get("https://www.melon.com/song/detail.htm?songId=" + address)
            lyric = driver.find_element_by_class_name("lyric").text
            LYRIC.append(lyric) 
        except:
            LYRIC.append("가사없음")
        time.sleep(4)
        
    return LYRIC


# 노래의 장르 반환
def genre(driver, song_address_li):
     
    GENRE = []
        
    for address in song_address_li[1:]:
        try:
            driver.get("https://www.melon.com/song/detail.htm?songId=" + address)
            genre = driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text
            GENRE.append(genre)
        except:
            GENRE.append("장르없음")
        
        time.sleep(4)
        
    return GENRE



# DB에 Insert해줄 함수
def insert_item_one(mongo, dates, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].insert_one(dates).inserted_id
    return result



def insert_item_many(mongo, dates, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].insert_many(dates).inserted_ids
    return result

ㅁ

## 크롤링하는 드라이버 정의

In [4]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
options = webdriver.ChromeOptions()
        
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# options.add_argument('headless')

# 드라이버 지정
driver = webdriver.Chrome('chromedriver.exe',options=options)

# 19금 노래들도 있어 맨 처음 로그인을 해준다.
driver.get('https://member.melon.com/muid/web/login/login_informM.htm')

# 로그인 정보
username = 'ID'
password = 'PASSWORD'

element_id = driver.find_element_by_name("id")
element_id.send_keys(username)
element_password = driver.find_element_by_name("pwd")
element_password.send_keys(password)


driver.find_element_by_class_name("btn_login03").click()

time.sleep(3)

# 크롤링 시작
driver.get(music_url)

## 크롤링 작업 수행

In [5]:
# 저장할 서버 DB 주소
host = "IP"
port = "PORT"
mongo = MongoClient(host, int(port))

# 크롤링한 데이터들이 담길 리스트
lyric_li = []
singer_li = []
title_li = []
genre_li = []

# 페이지 주소를 저장할 변수 num
num = 1

# 실제 크롤링 되는 for문
try:
    for _ in tqdm(range(13)):
        next_page( driver= driver,np_url = np_url, page=num) # 페이지 이동
        title_li.append(find_title(driver= driver)) # 노래 제목
        singer_li.append(find_singer(driver= driver)) # 가수
        song_address_list = find_song_address(driver = driver) # 노래 주소 반환
        genre_li.append(genre(driver= driver, song_address_li= song_address_list)) # 장르 반환
        lyric_li.append(lyric_crawl(driver= driver, song_address_li= song_address_list)) # 노래 가사 반환
    
        num += 50
        
except Exception as ex:
    print('에러가 발생 했습니다', ex)
    pass

    
# 몽고 디비에 입력
for i in range(len(title_li)):
    for j in range(len(title_li[0])):
        insert_item_one(mongo, {"genre": "{}".format(genre_li[i][j]), "title": "{}".format(title_li[i][j]), "singer" : "{}".format(singer_li[i][j]), "lyric" : "{}".format(lyric_li[i][j])},"Database", "Collection")
        
        

1page
2page
3page
4page
5page
6page
7page
8page
9page
10page
11page
12page
13page
